In [2]:
import sys
sys.path.append("/Colorization/src")  # Append path to src
import time

In [3]:
from util import *
import torch
import torchvision.datasets as dset
import torchvision.transforms as transforms

def input_trans(img_rgb_orig, HW=(256,256), resample=3):
    # return original size L and resized L as torch Tensors
    img_rgb_rs = resize_img(img_rgb_orig, HW=HW, resample=resample)
    
    img_lab_rs = color.rgb2lab(img_rgb_rs)

    img_l_rs = img_lab_rs[:,:,0]
    img_ab_rs = img_lab_rs[:,:,1:]

    tens_rs_l = torch.Tensor(img_l_rs)[None,None,:,:]
    tens_rs_ab = torch.Tensor(img_ab_rs)[None,None,:,:]

    return (tens_rs_l, tens_rs_ab)

In [15]:
def input_transforms(img_rgb_orig, target, HW=(256,256), resample=3):
    # return original size L and resized L as torch Tensors
    img_rgb_rs = resize_img(img_rgb_orig, HW=HW, resample=resample)
    
    img_lab_rs = color.rgb2lab(img_rgb_rs)

    img_l_rs = img_lab_rs[:,:,0]
    img_ab_rs = np.moveaxis(img_lab_rs[:,:,1:], -1, 0)  # (2, 256, 256)

    tens_rs_l = torch.Tensor(img_l_rs)[None,None,:,:]
    tens_rs_ab = torch.Tensor(img_ab_rs)[None,None,:,:]

    return (tens_rs_l, tens_rs_ab)

trainset = dset.CocoDetection(root='/Colorization/data/train2017', 
                              annFile='/Colorization/data/annotations/instances_train2017.json', 
                              transforms=input_transforms)

loading annotations into memory...
Done (t=9.49s)
creating index...
index created!


In [16]:
trainset[0][1].shape

torch.Size([1, 1, 2, 256, 256])

In [1]:
from util import *
from eccv16 import eccv16
import torch
import torch.utils.data as data
import torchvision.datasets as dset
import torchvision.transforms as transforms

def input_transforms(img_rgb_orig, target, HW=(256,256), resample=3):
    # return original size L and resized L as torch Tensors
    img_rgb_rs = resize_img(img_rgb_orig, HW=HW, resample=resample)
    
    img_lab_rs = color.rgb2lab(img_rgb_rs)

    img_l_rs = img_lab_rs[:,:,0]
    img_ab_rs = np.moveaxis(img_lab_rs[:,:,1:], -1, 0)  # (2, 256, 256)

    tens_rs_l = torch.Tensor(img_l_rs)[None,:,:]
    tens_rs_ab = torch.Tensor(img_ab_rs)[:,:]

    return (tens_rs_l, tens_rs_ab)

trainset = dset.CocoDetection(root='/Colorization/data/train2017', 
                              annFile='/Colorization/data/annotations/instances_train2017.json', 
                              transforms=input_transforms)

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-ke0y8yrz because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


loading annotations into memory...
Done (t=10.06s)
creating index...
index created!


In [2]:
pytorch_device = torch.device("cuda:0")#"cuda:0" if torch.cuda.is_available() else "cpu")

In [21]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64)

model = eccv16(pretrained=False).to(pytorch_device)

criterion = torch.nn.MSELoss(reduction='sum')
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
optimizer = torch.optim.Adam(model.parameters(), lr=3*1e-5,betas=(0.9,0.99),weight_decay=1e-3)

In [23]:
import time

tic = time.time()
for i, (img_l, y) in enumerate(trainloader):
    img_l = img_l.to(pytorch_device)
    y = y.to(pytorch_device)
    y_pred = model(img_l)
    print(np.linalg.norm(y_pred.detach().cpu().numpy() - y.cpu().numpy()))

    # Compute and print loss
    loss = criterion(y_pred, y)
    #loss = criterion(model.normalize_ab(y_pred), model.normalize_ab(y))
    
    toc = time.time()
    print(i, loss.item(), f'Time: {toc-tic} sec')
    tic = time.time()

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

48758.016
0 2377514240.0 Time: 2.050006151199341 sec
45549.938
1 2074922240.0 Time: 6.8503007888793945 sec
50817.266
2 2582576384.0 Time: 6.615176677703857 sec


KeyboardInterrupt: 

In [9]:
l, ab = trainset[0]
l.shape, ab.shape

(torch.Size([1, 256, 256]), torch.Size([2, 256, 256]))

In [12]:
ab_pred = model.to('cpu')(l[None,...])
ab_pred.shape

torch.Size([1, 2, 256, 256])

In [16]:
model.normalize_ab(ab), model.normalize_ab(ab_pred[0])

(tensor([[[ 2.7890e-01,  2.9074e-01,  2.9589e-01,  ..., -5.4961e-02,
           -5.7423e-02, -5.7302e-02],
          [ 2.9895e-01,  3.0024e-01,  3.0319e-01,  ..., -4.4786e-02,
           -5.7571e-02, -5.7384e-02],
          [ 2.9119e-01,  2.8268e-01,  2.8741e-01,  ..., -5.5058e-02,
           -6.5139e-02, -6.5045e-02],
          ...,
          [ 1.8206e-02,  2.3610e-02,  7.9364e-03,  ...,  1.4986e-02,
           -1.4619e-02,  4.0481e-02],
          [ 1.3301e-02,  2.0583e-02,  1.3905e-02,  ...,  2.1085e-02,
            4.0446e-02,  5.0372e-02],
          [ 1.3301e-02,  1.9319e-02,  2.5926e-02,  ...,  6.3628e-02,
            5.7654e-02,  3.9001e-02]],
 
         [[-4.6523e-01, -4.7776e-01, -4.8582e-01,  ..., -1.5565e-01,
           -1.5721e-01, -1.5762e-01],
          [-4.7971e-01, -4.8441e-01, -4.9320e-01,  ..., -1.5734e-01,
           -1.5667e-01, -1.5735e-01],
          [-4.7720e-01, -4.7501e-01, -4.8361e-01,  ..., -1.5526e-01,
           -1.5364e-01, -1.5402e-01],
          ...,
    

In [14]:
ab_pred[0], ab

(tensor([[[ 12.1945,  12.1945,  11.1531,  ...,  -1.1940,  -1.2353,  -1.2353],
          [ 12.1945,  12.1945,  11.1531,  ...,  -1.1940,  -1.2353,  -1.2353],
          [ 11.0493,  11.0493,  10.1825,  ...,  -1.1962,  -1.2432,  -1.2432],
          ...,
          [  2.0328,   2.0328,   2.0499,  ...,  11.4133,  10.7650,  10.7650],
          [  2.1856,   2.1856,   2.1696,  ...,   9.5490,   8.7544,   8.7544],
          [  2.1856,   2.1856,   2.1696,  ...,   9.5490,   8.7544,   8.7544]],
 
         [[ -2.3308,  -2.3308,  -4.6468,  ...,  -8.1821,  -8.2327,  -8.2327],
          [ -2.3308,  -2.3308,  -4.6468,  ...,  -8.1821,  -8.2327,  -8.2327],
          [ -3.4979,  -3.4979,  -5.5311,  ...,  -8.0524,  -8.1456,  -8.1456],
          ...,
          [ -3.8110,  -3.8110,  -4.3346,  ..., -10.0400,  -9.7794,  -9.7794],
          [ -2.9274,  -2.9274,  -3.4597,  ...,  -8.9700,  -8.4904,  -8.4904],
          [ -2.9274,  -2.9274,  -3.4597,  ...,  -8.9700,  -8.4904,  -8.4904]]],
        grad_fn=<SelectBackwa